In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

def preprocess_and_resize_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

def generate_grad_cam_heatmap(image_path, target_size, class_index):
    preprocessed_image = preprocess_and_resize_image(image_path, target_size)
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(preprocessed_image)
        loss = predictions[:, class_index]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap

def visualize_grad_cam_and_save(image_path, target_size, heatmap, output_dir, alpha=0.5):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)

    # Convert the heatmap to a 3-channel image
    heatmap = np.uint8(255 * heatmap)
    jet = plt.get_cmap('jet')
    jet_colors = jet(heatmap)
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_colors)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = img_to_array(jet_heatmap)
    jet_heatmap = jet_heatmap[:, :, :3]  # Keep only the RGB channels

    # Overlay the heatmap on the original image
    superimposed_img = alpha * jet_heatmap + (1 - alpha) * img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the image with heatmap overlay
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    superimposed_img.save(output_path)

    # Display the image (optional)
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Grad-CAM

model = VGG16(weights='imagenet', include_top=True)
last_conv_layer_name = 'block5_conv3'
last_conv_layer = model.get_layer(last_conv_layer_name)
grad_model = tf.keras.models.Model([model.inputs], [last_conv_layer.output, model.output])

gradcam_dir = '/content/drive/MyDrive/images'
output_dir = '/content/drive/MyDrive/heatmaps'
class_labels = ['a', 'b', 'c', 'd', 'e', 'f']
target_size = (224, 224)

os.makedirs(output_dir, exist_ok=True)

for class_idx, class_label in enumerate(class_labels):
    class_dir = os.path.join(gradcam_dir, class_label)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        heatmap = generate_grad_cam_heatmap(image_path, target_size, class_idx)
        visualize_grad_cam_and_save(image_path, target_size, heatmap, output_dir)